## Matrix multiplication from foundations

In [1]:
from pathlib import Path
import matplotlib as mpl, matplotlib.pyplot as plt

In [2]:
import torch
from torch import tensor

## Element wise Ops

In [3]:
a = tensor([10., 6, -4])
b = tensor([2., 8, 7])
a,b

(tensor([10.,  6., -4.]), tensor([2., 8., 7.]))

In [4]:
a + b

tensor([12., 14.,  3.])

In [5]:
(a < b).float().mean()

tensor(0.6667)

In [6]:
m = tensor([[1., 2, 3], [4,5,6], [7,8,9]])
print(m)

tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])


In [7]:
sf = (m*m).sum()
sf

tensor(285.)

In [8]:
sf.sqrt()

tensor(16.8819)

In [9]:
m[2,:],m[:,2]

(tensor([7., 8., 9.]), tensor([3., 6., 9.]))

In [10]:
m[2]

tensor([7., 8., 9.])

## Broadcasting

The term **broadcasting** describes how arrays with different shapes are treated during arithmetic operations.

From the [Numpy Documentation](https://docs.scipy.org/doc/numpy-1.10.0/user/basics.broadcasting.html):

    The term broadcasting describes how numpy treats arrays with 
    different shapes during arithmetic operations. Subject to certain 
    constraints, the smaller array is “broadcast” across the larger 
    array so that they have compatible shapes. Broadcasting provides a 
    means of vectorizing array operations so that looping occurs in C
    instead of Python. It does this without making needless copies of 
    data and usually leads to efficient algorithm implementations.
    
In addition to the efficiency of broadcasting, it allows developers to write less code, which typically leads to fewer errors.

*This section was adapted from [Chapter 4](http://nbviewer.jupyter.org/github/fastai/numerical-linear-algebra/blob/master/nbs/4.%20Compressed%20Sensing%20of%20CT%20Scans%20with%20Robust%20Regression.ipynb#4.-Compressed-Sensing-of-CT-Scans-with-Robust-Regression) of the fast.ai [Computational Linear Algebra](https://github.com/fastai/numerical-linear-algebra) course.*

### Broadcasting with a scalar

In [15]:
a

tensor([10.,  6., -4.])

In [16]:
a > 0

tensor([ True,  True, False])

How are we able to do `a > 0`?  0 is being **broadcast** to have the same dimensions as a.

For instance you can normalize our dataset by subtracting the mean (a scalar) from the entire data set (a matrix) and dividing by the standard deviation (another scalar), using broadcasting.

Other examples of broadcasting with a scalar:

In [17]:
a + 1

tensor([11.,  7., -3.])

In [18]:
m

tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])

In [19]:
2*m

tensor([[ 2.,  4.,  6.],
        [ 8., 10., 12.],
        [14., 16., 18.]])

### Broadcasting a vector to a matrix

Although broadcasting a scalar is an idea that dates back to A Programming Language, the more powerful idea of broadcasting across higher rank tensors [comes from](https://mail.python.org/pipermail/matrix-sig/1995-November/000143.html) a little known language called [Yorick](https://software.llnl.gov/yorick-doc/manual/yorick_50.html).

We can also broadcast a vector to a matrix:

In [20]:
c = tensor([10.,20,30]); c

tensor([10., 20., 30.])

In [21]:
m

tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])

In [22]:
m.shape,c.shape

(torch.Size([3, 3]), torch.Size([3]))

In [23]:
m + c

tensor([[11., 22., 33.],
        [14., 25., 36.],
        [17., 28., 39.]])

In [24]:
c + m

tensor([[11., 22., 33.],
        [14., 25., 36.],
        [17., 28., 39.]])

In [25]:
t = c.expand_as(m)

In [31]:
t

tensor([[10., 20., 30.],
        [10., 20., 30.],
        [10., 20., 30.]])

In [32]:
m + t

tensor([[11., 22., 33.],
        [14., 25., 36.],
        [17., 28., 39.]])

We don't really copy the rows, but it looks as if we did. In fact, the rows are given a *stride* of 0.

In [35]:
t.storage()

 10.0
 20.0
 30.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 3]

In [36]:
t.stride(), t.shape

((0, 1), torch.Size([3, 3]))

You can index with the special value [None] or use `unsqueeze()` to convert a 1-dimensional array into a 2-dimensional array (although one of those dimensions has value 1).

In [37]:
c.unsqueeze(0), c[None, :]

(tensor([[10., 20., 30.]]), tensor([[10., 20., 30.]]))

In [38]:
c.shape, c.unsqueeze(0).shape

(torch.Size([3]), torch.Size([1, 3]))

In [ ]:
c.unsqueeze(1), c[:, None]

(tensor([[10.],
         [20.],
         [30.]]),
 tensor([[10.],
         [20.],
         [30.]]))

In [ ]:
c.shape, c.unsqueeze(1).shape

(torch.Size([3]), torch.Size([3, 1]))

You can always skip trailling ':'s. And '...' means '*all preceding dimensions*'

In [39]:
c[None].shape,c[...,None].shape

(torch.Size([1, 3]), torch.Size([3, 1]))

In [40]:
c[:,None].expand_as(m)

tensor([[10., 10., 10.],
        [20., 20., 20.],
        [30., 30., 30.]])

In [41]:
m + c[:,None]

tensor([[11., 12., 13.],
        [24., 25., 26.],
        [37., 38., 39.]])

In [42]:
m + c[None,:]

tensor([[11., 22., 33.],
        [14., 25., 36.],
        [17., 28., 39.]])

### Broadcasting Rules

In [43]:
c[None,:]

tensor([[10., 20., 30.]])

In [44]:
c[None,:].shape

torch.Size([1, 3])

In [45]:
c[:,None]

tensor([[10.],
        [20.],
        [30.]])

In [46]:
c[:,None].shape

torch.Size([3, 1])

In [47]:
c[None,:] * c[:,None]

tensor([[100., 200., 300.],
        [200., 400., 600.],
        [300., 600., 900.]])

In [48]:
c[None] > c[:,None]

tensor([[False,  True,  True],
        [False, False,  True],
        [False, False, False]])

In [49]:
m*m

tensor([[ 1.,  4.,  9.],
        [16., 25., 36.],
        [49., 64., 81.]])

When operating on two arrays/tensors, Numpy/PyTorch compares their shapes element-wise. It starts with the **trailing dimensions**, and works its way forward. Two dimensions are **compatible** when

- they are equal, or
- one of them is 1, in which case that dimension is broadcasted to make it the same size

Arrays do not need to have the same number of dimensions. For example, if you have a `256*256*3` array of RGB values, and you want to scale each color in the image by a different value, you can multiply the image by a one-dimensional array with 3 values. Lining up the sizes of the trailing axes of these arrays according to the broadcast rules, shows that they are compatible:

    Image  (3d array): 256 x 256 x 3
    Scale  (1d array):             3
    Result (3d array): 256 x 256 x 3

The [numpy documentation](https://docs.scipy.org/doc/numpy-1.13.0/user/basics.broadcasting.html#general-broadcasting-rules) includes several examples of what dimensions can and can not be broadcast together.

## Matrix multiplications

In [52]:
m1 = torch.rand(5, 784)   # Tensor of size (5, 784)
m2 = torch.rand(784, 10)  # Tensor of size (784, 10)

In [53]:
m1.shape, m2.shape

(torch.Size([5, 784]), torch.Size([784, 10]))

## Matmul with broadcasting

In [54]:
digit = m1[0]
digit.shape,m2.shape

(torch.Size([784]), torch.Size([784, 10]))

In [55]:
digit[:,None].shape

torch.Size([784, 1])

In [56]:
digit[:,None].expand_as(m2).shape

torch.Size([784, 10])

In [57]:
(digit[:,None]*m2).shape

torch.Size([784, 10])

In [79]:
def matmul(a,b):
    (ar,ac),(br,bc) = a.shape,b.shape
    c = torch.zeros(ar, bc)
    for i in range(ar):
#       c[i,j] = (a[i,:] * b[:,j]).sum()      # regular version
        c[i]   = (a[i,:,None] * b).sum(dim=0) # broadcast version
    return c

In [80]:
%timeit -n 50 _=matmul(m1, m2)

190 µs ± 112 µs per loop (mean ± std. dev. of 7 runs, 50 loops each)


<500µs, quite fast !

## pytorch op

We can use pytorch's function or operator directly for matrix multiplication.

In [81]:
%timeit -n 5 _=torch.matmul(m1, m2)

The slowest run took 24.27 times longer than the fastest. This could mean that an intermediate result is being cached.
69 µs ± 129 µs per loop (mean ± std. dev. of 7 runs, 5 loops each)


## CUDA

In [84]:
m1c,m2c = m1.cuda(),m2.cuda()

In [85]:
r=(m1c@m2c).cpu()

In [86]:
%timeit -n 10 r=(m1c@m2c).cpu()

43 µs ± 28.8 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Other operations
Now create matrices and play around with the following operations: 
- reshaping: tensor.view(new_shape) or tensor.reshape(new_shape)
- concatenation: torch.cat([tensor1, tensor2], dim=0)
- stacking: torch.stack([tensor1, tensor2], dim=0)
- argmax: torch.argmax(tensor)
- transpose: torch.transpose(tensor, dim0, dim1)
- permute: tensor.permute(dim0, dim1, ...)

Don't forget to checkout [pytorch documentation!](https://pytorch.org/docs/2.1/)